In [1]:
import sqlite3 as sq3
import csv


In [2]:
## Constant definitions
DBNAME = "Gold_Country_OSM2.db"
## CSV files to load into SQLite database
CSVFILES = ['users.csv', 'nodes.csv','ways.csv','relations.csv','node_tags.csv',\
           'way_tags.csv','relation_tags.csv','way_nodes.csv',\
           'relation_members.csv']


## SQL Insert statements for loading data
INSQL = ["""INSERT INTO Users (uid, user) VALUES (:uid, :user);""",
        """INSERT INTO Nodes (id, lat, lon, uid, version, changeset, timestamp)
        VALUES (:id, :lat, :lon, :uid, :version, :changeset, :timestamp)""", \
        """INSERT INTO Ways (id, uid, version, changeset, timestamp)
        VALUES (:id, :uid, :version, :changeset, :timestamp)""", \
        """INSERT INTO Relations (id, uid, version, changeset, timestamp)
        VALUES (:id, :uid, :version, :changeset, :timestamp)""", \
        """INSERT INTO Node_Tags (id, key, value, type)
        VALUES (:id, :key, :value, :type)""", \
        """INSERT INTO Way_Tags (id, key, value, type)
        VALUES (:id, :key, :value, :type)""", \
        """INSERT INTO Relation_Tags (id, key, value, type)
        VALUES (:id, :key, :value, :type)""", \
        """INSERT INTO Way_Nodes (id, node_id, position)
        VALUES (:id, :node_id, :position)""", \
        """INSERT INTO Relation_Members (id, member_id, role, type, position)
        VALUES (:id, :member_id, :role, :type, :position)"""]

## SQL Table create statements
TCSQL = ["""CREATE TABLE Users(
        uid INT PRIMARY KEY,
        user TEXT);""",
        """CREATE TABLE Ways( 
        id INT PRIMARY KEY,  
        uid INT REFERENCES Users, 
        version TEXT, 
        changeset TEXT, 
        timestamp TEXT);""", \
        """CREATE TABLE Nodes(
        id INT PRIMARY KEY, 
        lat REAL, 
        lon REAL, 
        uid TEXT REFERENCES Users, 
        version TEXT, 
        changeset TEXT, 
        timestamp TEXT);""", \
        """CREATE TABLE Relations( 
        id INT PRIMARY KEY, 
        uid INT REFERENCES Users, 
        version TEXT, 
        changeset TEXT, 
        timestamp TEXT);""", \
        """CREATE TABLE Way_Nodes ( 
        id INT REFERENCES Ways, 
        node_id INT REFERENCES Nodes(id), 
        position INT, 
        PRIMARY KEY (id, node_id, position));""", \
        """CREATE TABLE Way_Tags ( 
        id INT REFERENCES Ways, 
        key TEXT, 
        value TEXT, 
        type TEXT, 
        PRIMARY KEY(id,key,type));""", \
        """CREATE TABLE Node_Tags ( 
        id INT REFERENCES Nodes, 
        key TEXT, 
        value TEXT, 
        type TEXT, 
        PRIMARY KEY(id,key,type));""", \
        """CREATE TABLE Relation_Members( 
        id INT REFERENCES Relations, 
        member_id INT, 
        role TEXT, 
        type TEXT, 
        position INT, 
        PRIMARY KEY(id,member_id,position));""", \
        """CREATE TABLE Relation_tags( 
        id INT REFERENCES Relations, 
        key TEXT, 
        value TEXT, 
        type TEXT, 
        PRIMARY KEY(id,key,type));"""]


In [3]:
def UnicodeDictReader(utf8_data, **kwargs):
    csv_reader = csv.DictReader(utf8_data, **kwargs)
    for row in csv_reader:
        yield {key: unicode(value, 'utf-8') for key, value in row.iteritems()}

In [4]:
## Execute the CREATE TABLE statements for the database

with sq3.connect(DBNAME) as conn:
    for sql in TCSQL:
        conn.execute(sql)
print 'done'

done


In [5]:
## Create the dictionary of csv files and SQL INSERT statements:
populate_tables = dict(zip(CSVFILES,INSQL))


In [6]:
## Read each csv file and insert values into appropriate table in SQLite DB file.
with sq3.connect(DBNAME) as conn:
    cur = conn.cursor()
    for csvfile in populate_tables.keys():
        with open (csvfile, 'rb') as infile:
            csv_reader = UnicodeDictReader(infile)
            cur.executemany(populate_tables[csvfile], csv_reader)
            conn.commit()
print "done"    

done
